# CA02: This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm. 

In this assignment you will ...
1. Complete the code such a way that it works correctly with this given parts of the program.
2. Explain as clearly as possible what each part of the code is doing. Use "Markdown" texts and code commenting to explain the code

__IMPORTANT NOTE:__

The path of your data folders 'train-mails' and 'test-mails' must be './train-mails' and './test-mails'. This means you must have your .ipynb file and these folders in the SAME FOLDER in your laptop or Google Drive. The reason for doing this is, this way the peer reviewes and I would be able to run your code from our computers using this exact same relative path, irrespective of our folder hierarchy.

In [1]:
import os
import numpy as np
from collections import Counter

# Import all other necessary libraries. Your code below ...
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import itertools

In [2]:
import pkg_resources

def print_versions():
    packages = ['os', 'numpy', 'collections', 'pandas', 'sklearn', 'matplotlib', 'itertools', 'pkg_resources']
    for package in packages:
        try:
            package_version = pkg_resources.get_distribution(package).version
            print(f'{package}=={package_version}')
        except:
            print(f'{package} is not installed')

print_versions()

os is not installed
numpy==1.22.1
collections is not installed
pandas==1.4.0
sklearn is not installed
matplotlib==3.5.1
itertools is not installed
pkg_resources is not installed


---

This code defines a function `make_Dictionary` that takes a root directory `(root_dir)` as input. The purpose of this function is to create a dictionary of most common words from a set of text files stored in the specified root directory.

The steps performed by this function are as follows:

1. Initialize an empty list `all_words` to store all words from the text files.
2. Create a list of email file paths (`emails`) by joining the root_dir and filenames obtained by calling `os.listdir(root_dir)`.
3. For each email file path in `emails`, open the file and read its content line by line.
4. For each line in the email file, split the line into words using the `.split()` method, and add these words to the `all_words` list.
5. Create a dictionary `dictionary` using the `Counter` method, which will store the count of each word in the `all_words` list.
6. Create a list `list_to_remove` containing all the keys in the dictionary.
7. Loop through `list_to_remove`, and remove the keys that are either not alphabetic or have only one character.
8. Retain the 3000 most common words in the dictionary using the `most_common` method, and return the dictionary.

In [3]:
# Create a dictionary 
def make_Dictionary(root_dir):
    
  all_words = []
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
    
  for mail in emails:
    with open(mail) as m:
      for line in m:
        words = line.split()
        all_words += words
        
  dictionary = Counter(all_words)
  list_to_remove = list(dictionary)

  for item in list_to_remove:
    if item.isalpha() == False:
      del dictionary[item]
    elif len(item) == 1:
      del dictionary[item]
    
  dictionary = dictionary.most_common(3000)
  return dictionary

---

This code defines a function `extract_features` that takes a directory `(mail_dir)` as input. The purpose of this function is to extract features from text files stored in the specified directory.

The steps performed by this function are as follows:

1. Create a list `files` of file paths by joining the mail_dir and filenames obtained by calling os.listdir(mail_dir).
2. Initialize a feature matrix `features_matrix` with dimensions `(len(files), 3000)` as a numpy array of zeros. This matrix will store the frequency of the most common 3000 words in each text file.
3. Initialize an array `train_labels` with length `len(files)` as a numpy array of zeros. This array will store the labels for each text file, where a label of 0 indicates a non-spam email and a label of 1 indicates a spam email.
4. Initialize a counter `count` to keep track of the number of spam emails, and initialize a variable `docID` to keep track of the current file index.
5. Loop through the `files` list, and for each file:
    * Open the file and read its content line by line.
    * For the 3rd line of the file, split the line into words using the `.split()` method, and for each word:
        * Initialize `wordID` to 0.
        * Loop through the dictionary list and check if the word is present in the dictionary. If yes, set the   `wordID` to the index of the word in the dictionary and increment the corresponding entry in the `features_matrix` for the current file by the count of the word in the current line.
    * Set the label of the current file to 0 in the `train_labels` array.
    * If the last token of the file path starts with "`spmsg`", set the label to 1, increment the counter `count`, and update the `train_labels` array accordingly.
    * Increment the `docID` by 1.
6. Return the `features_matrix` and `train_labels` arrays.

In [4]:
def extract_features(mail_dir):
    
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  features_matrix = np.zeros((len(files),3000))
  train_labels = np.zeros(len(files))
  count = 1;
  docID = 0;

  for fil in files:
    with open(fil) as fi:
      for i, line in enumerate(fi):
        if i ==2:
          words = line.split()
          for word in words:
            wordID = 0
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                features_matrix[docID,wordID] = words.count(word)
      train_labels[docID] = 0;
      filepathTokens = fil.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):
        train_labels[docID] = 1;
        count = count + 1
      docID = docID + 1
    
  return features_matrix, train_labels                

---

This code defines two string variables `TRAIN_DIR` and `TEST_DIR` that represent the paths to two directories.

`TRAIN_DIR` is the path to the directory that contains training email data, while `TEST_DIR` is the path to the directory that contains test email data.

Both directories are part of a larger file system path, which is specified as:
`MSBA Machine Learning/Computer Assignments/CA02 - Spam eMail Detection using Naive Bayes Classification Algorithm/`

In [5]:
# Enter the "path" of your "train_mails" and "test-mails" FOLDERS in this cell...

# for example: 
TRAIN_DIR = './train-mails'
TEST_DIR = './test-mails'

#TRAIN_DIR = "/home/jupyter-svanhemert00/MSBA_ML/computer_assignments/CA02/train-mails"
#TEST_DIR = "/home/jupyter-svanhemert00/MSBA_ML/computer_assignments/CA02/test-mails"

This code calls two functions, `make_Dictionary` and `extract_features`, and uses their returned values to create several variables.

`make_Dictionary` function is called with `TRAIN_DIR` as the input, which is a directory path to the training emails. This function returns a dictionary object, which is a list of the most common 3000 words found in the emails in the training directory.

`extract_features` function is then called twice, once with `TRAIN_DIR` and once with `TEST_DIR`, as the inputs. This function returns two arrays, `features_matrix` and `labels` for the training directory, and `test_features_matrix` and `test_labels` for the test directory. These arrays represent the features and labels of the emails in the directories, respectively.

The output "`reading and processing emails from TRAIN and TEST folders`" is just a message to the user to indicate that the code is currently processing the emails from both the training and test folders.

In [6]:
dictionary = make_Dictionary(TRAIN_DIR)

print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(TRAIN_DIR)
test_features_matrix, test_labels = extract_features(TEST_DIR)

reading and processing emails from TRAIN and TEST folders


In [7]:
print('features_matrix:')
print(features_matrix)
print()
print('test_features_matrix:')
print(test_features_matrix)
print()
print('labels:')
print(labels)
print()
print('test_labels:')
print(test_labels)

features_matrix:
[[5. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]
 [3. 0. 2. ... 0. 0. 0.]]

test_features_matrix:
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

labels:
[1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1.
 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0.
 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0.
 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1.
 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0.
 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 

---

# Machine Learning
In this section enter your code to TRAIN the model using Naive Bayes algorithm, then PREDICT and then evaluate PERFORMANCE (Accuracy)

Your code below ...

## Select an ML Algorithm

In [8]:
# importing the Naive Bayes algorithm module Gaussian which is used in classification
# it assumes that features follow normal distribution
from sklearn.naive_bayes import GaussianNB 

# using Gaussian Naive Bayes Algorithm
model = GaussianNB()

## Train the model using Training Dataset "Features `(X_train)`" and "Outcome `(y_train)`"

In [9]:
# training Naive bayes classifier
print ("Training Model using Gaussian Naive Bayes algorithm .....")
model.fit(features_matrix, labels)
print ("Training completed")

Training Model using Gaussian Naive Bayes algorithm .....
Training completed


## Test the trained model by predicting from the Test Data "Features (X_test)"

In [10]:
# test the unseen passangers from test dataset
print ("testing trained model to predict Test Data labels")
y_predicted = model.predict(test_features_matrix)
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")

testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:


In [11]:
# here, I wanted to plot the naive bayes model using a matplotlib scatterplot
# this code was generated using ChatGPT

'''
# Plot the decision boundary
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.5)

# Plot the data points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.get_cmap('coolwarm', 2))
plt.show()

# This will generate a scatter plot of the data points colored according to the class labels,
# and the decision boundary of the model as a contour plot.
'''

"\n# Plot the decision boundary\nx_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1\ny_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1\nxx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))\nZ = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]\nZ = Z.reshape(xx.shape)\nplt.contourf(xx, yy, Z, alpha=0.5)\n\n# Plot the data points\nplt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.get_cmap('coolwarm', 2))\nplt.show()\n\n# This will generate a scatter plot of the data points colored according to the class labels,\n# and the decision boundary of the model as a contour plot.\n"

If you have a 2-class Naive Bayes model trained on 2-dimensional data, you can create a scatter plot of the data points and color-code them based on the class labels. Then you can plot the decision boundary of the model as a contour line separating the two classes.

## Calculate the performance by comparing `y_predicted` against `test_labels`

In [12]:
# print accuracy of the model

# in multilabel classification, this function computes subset accuracy
from sklearn.metrics import accuracy_score, classification_report

# accuracy score is just percentage of correct predictions
print(' Test Labels accuracy_score:',accuracy_score(test_labels, y_predicted))
print()

 Test Labels accuracy_score: 0.9615384615384616



In [15]:
# evaluation metrics

# download evaluation metrics from Scikit-Learn
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Calculate accuracy: in multilabel classification, this function computes subset accuracy
accuracy = accuracy_score(test_labels, y_predicted)
print("Accuracy:", accuracy)
print()

# Generate the confusion matrix
confusion_matrix = confusion_matrix(test_labels, y_predicted)
print("Confusion Matrix:\n", confusion_matrix)
print()

# Generate the classification report
class_report = classification_report(test_labels, y_predicted)
print("Classification Report:\n", class_report)

Accuracy: 0.9615384615384616

Confusion Matrix:
 [[129   1]
 [  9 121]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.93      0.99      0.96       130
         1.0       0.99      0.93      0.96       130

    accuracy                           0.96       260
   macro avg       0.96      0.96      0.96       260
weighted avg       0.96      0.96      0.96       260



In [14]:
# here, I wanted to plot the correlation matrix using a matplotlib heatmap, but I am having trouble doing so.
# this code was generated using ChatGPT

'''
plt.imshow(confusion_matrix, cmap=plt.cm.Blues)
plt.colorbar()
plt.xlabel("Predicted Class")
plt.ylabel("True Class")

classes = np.unique(test_labels)
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, y_predicted, rotation=45)
plt.yticks(tick_marks, test_labels)

thresh = confusion_matrix.max() / 2.
for i, j in itertools.product(range(confusion_matrix.shape[0]), range(confusion_matrix.shape[1])):
    plt.text(j, i, confusion_matrix[i, j],
             horizontalalignment="center",
             color="white" if confusion_matrix[i, j] > thresh else "black")

plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()
'''

'\nplt.imshow(confusion_matrix, cmap=plt.cm.Blues)\nplt.colorbar()\nplt.xlabel("Predicted Class")\nplt.ylabel("True Class")\n\nclasses = np.unique(test_labels)\ntick_marks = np.arange(len(classes))\nplt.xticks(tick_marks, y_predicted, rotation=45)\nplt.yticks(tick_marks, test_labels)\n\nthresh = confusion_matrix.max() / 2.\nfor i, j in itertools.product(range(confusion_matrix.shape[0]), range(confusion_matrix.shape[1])):\n    plt.text(j, i, confusion_matrix[i, j],\n             horizontalalignment="center",\n             color="white" if confusion_matrix[i, j] > thresh else "black")\n\nplt.title("Confusion Matrix")\nplt.tight_layout()\nplt.show()\n'

# Conclusion
In conclusion, we were able to create a notebook that successfully detects spam emails from a normal emails at 96% accuracy. In doing so, we created functions that read train and text files from our directory and pre-processed the data by removing stopwords, non-alphabetical characters, one-letter words, and by tokenizing and vectorizing the remaining words. After creating our train and test features and labels, we used the Scikit-Learn library to run a Naive Bayes classification model on our data and print evaluation metrics. 

The evaluation metrics indicate that:
- __Accuracy__: 96% of our predictions match that of `y_test`
- We had 1 false positive and 9 false negatives, meaning that 1 non-spam email was labeled as spam and 9 spam emails were labeled as non-spam. In this use-case, I believe Type I Statistical Errors present a larger risk given that one might be expecting an important, clean email that is no where to be found because it was automatically sent to the spam folder.
- __Precision__: 99% indicates that 129 out of 130 of actual positives were detected
- __Recall__: 93% indicates out of the ones called positive, 93% of them were correct

There are other evaluation metrics that could have been used and I still need help understanding these metrics thouroughly

In the future, a good idea would have been to conduct lemmatization or stemming on our features (words)

======================= END OF PROGRAM =========================